In [144]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Load the train data set
train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")

print(train.shape)
print(test.shape)


(891, 12)
(418, 11)


#### Training Data

In [150]:
# Load the train data set
train = pd.read_csv('train.csv')

# Can simplify the Titles with this dictionary:
Title_Dictionary = {"Capt"  : "Officer", 
                    "Col"   : "Officer",
                    "Major" : "Officer",
                    "Dr"    : "Officer", 
                    "Rev"   : "Officer", 
                    "Don"   : "Royalty", 
                    "Sir"   : "Royalty", 
                    "Jonkheer": "Royalty",
                    "the Countess":"Royalty", 
                    "Mr"    : "Mr", 
                    "Mme"   : "Mrs", 
                    "Mrs"   : "Mrs", 
                    "Ms"    : "Miss", 
                    "Miss"  : "Miss", 
                    "Mlle"  : "Miss", 
                    "Master" : "Master", 
                    "Lady"  : "Royalty" 
                    }
    
# adding summarized "Title" column to data
train["Title"] = train['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
train["Title"] = train["Title"].map(Title_Dictionary)
print(train["Title"].value_counts())

from sklearn.preprocessing import OneHotEncoder

for i in ('Sex','Embarked', 'Title'):
    # Create a one-hot encoder
    encoder = OneHotEncoder(handle_unknown='ignore')

    # Encode the "Sex" column
    encoder.fit(train[[i]])
    sex_encoded = encoder.transform(train[[i]])
    sex_encoded_df = pd.DataFrame(sex_encoded.toarray(), columns=encoder.get_feature_names_out([i]))

    # Join the encoded column to the original dataframe
    train = pd.concat([train, sex_encoded_df], axis=1)

    # Drop the original "Sex" column
    train = train.drop(columns=[i])
    # print(train.head(2))

# train.head(3)

# NOTE Could also do it this way
# from sklearn.preprocessing import LabelEncoder
# # One limitations of this: it changes the column categorical values into numerical values, but then the model may interpret these numerical values as 
# # being ordinal.
# # If the data is not ordered (as in  the Sex here, there's no reason that male should be 1 and female 0, or vice versa) you should
# # use OneHot encoder instead

# # Create a label encoder
# encoder = LabelEncoder()

# # Encode the non-numeric columns
# train['Sex'] = encoder.fit_transform(train['Sex']

# train['Embarked'] = encoder.fit_transform(train['Sex'])
# train.head()
# train["Embarked"].value_counts()


# Drop the unnecessary columns, then drop the NA values

train = train.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'])
# print(train.head())
train = train.dropna()
print(train.shape)

# Split the data into X and y
x_train = train.drop(columns=['Survived'])
y_train = train['Survived']

# x_train = x_train.dropna()
# x_train = x_train.dropna()

print(f"x_train shape = {x_train.shape}")
print(f"y_train shape = {y_train.shape}")

# ----------------------------------------------------------------------
# CREATING THE MODEL: Sklearn Random Forest Classifier 
# ----------------------------------------------------------------------

# Create a random forest classifier
rf = RandomForestClassifier(random_state=1)

# Use RFE (Recursive Factor Elimination) to select the top 4 features
my_RF_model = RFE(estimator=rf, n_features_to_select=4)
my_RF_model.fit(x_train, y_train)

# Print the selected features
print(x_train.columns[my_RF_model.support_])

# print(x_train.columns)
print(my_RF_model.ranking_)


Mr         517
Miss       185
Mrs        126
Master      40
Officer     18
Royalty      5
Name: Title, dtype: int64
(714, 18)
x_train shape = (714, 17)
y_train shape = (714,)
Index(['Age', 'Fare', 'Sex_female', 'Sex_male'], dtype='object')
[ 2  1  4  5  1  1  1  7 12  9 14 10  6  3  8 11 13]


#### "Test" Data

In [147]:
# NOTE: DOING IT ALL AGAIN FOR TEST DATA:

# Load the train data set
test = pd.read_csv("test.csv")
print(f"Original Test shape = {test.shape}")


# adding summarized "Title" column to data
test["Title"] = test['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
test["Title"] = test["Title"].map(Title_Dictionary)
# print(test["Title"].value_counts())

from sklearn.preprocessing import OneHotEncoder

for i in ('Sex','Embarked', 'Title'):
    # Create a one-hot encoder
    encoder = OneHotEncoder(handle_unknown='ignore')

    # Encode the current column
    encoder.fit(test[[i]])
    sex_encoded = encoder.transform(test[[i]])
    sex_encoded_df = pd.DataFrame(sex_encoded.toarray(), columns=encoder.get_feature_names_out([i]))

    # Join the encoded column to the original dataframe
    test = pd.concat([test, sex_encoded_df], axis=1)

    # Drop the original column
    test = test.drop(columns=[i])
    # print(train.head(2))

test = test.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'])



# print(test.columns)
# column_list = x_train.columns[my_RF_model.support_]
# # print(x_train.columns[my_RF_model.support_])
# print(column_list)

# x_test = test[column_list]
# print(x_test.columns)




# test = test[my_RF_model.support_]
# test.columns
# test = test.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'])

# print(test.head())
# print(f"Final Test shape = {test.shape}")
# print(f"y_test shape = {y_test.shape}")


Original Test shape = (418, 11)


In [119]:

# Create a random forest classifier
# rf = RandomForestClassifier(random_state=1)

# Use RFE to select the top 4 features

# Create a random forest classifier
# rf = RandomForestClassifier(random_state=1)

# # Use RFE (Recursive Factor Elimination) to select the top 4 features
# my_RF_model = RFE(estimator=rf, n_features_to_select=5)
# my_RF_model.fit(x_train, y_train)

# # Print the selected features
# print(x_train.columns[my_RF_model.support_])

# # print(x_train.columns)
# print(my_RF_model.ranking_)


#### Submission

In [148]:
# my_RF_model.fit(te, train_y)

y_predicted = my_RF_model.predict(test)
print(y_predicted)


/Users/Henry/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Title_nan
Feature names seen at fit time, yet now missing:
- Embarked_nan
- Title_Royalty

  warnings.warn(message, FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:

df = pd.DataFrame(y_predicted, columns = ['Survived'])
df.info()

In [ ]:
submission = pd.DataFrame({"PassengerId" : test['PassengerId'], "Survived" : y_predicted}).astype('int32')
# submission.info()
submission.to_csv("submission_RF.csv", index=False)